In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np
from scipy.stats import linregress
import bisect
from scipy import stats

from src.mathTools.SpecialMatrix import SpecialMatrix
from src.mathTools.DistributionTools import DistributionTools


In [5]:
df = pl.DataFrame(
    {
        "foo": [1, 2, 3],
        "bar": [6.0, 7.0, 8.0],
        "ham": ["a", "b", "c"],
    }
)

other_df = pl.DataFrame(
    {
        "apple": ["x", "y", "z"],
        "ham": ["a", "b", "d"],
        "bar": [6.0, -2.0, -3.0],
    }
)
df.join(other_df, on="ham", how="full", coalesce=True, suffix="_other")

foo,bar,ham,apple,bar_other
i64,f64,str,str,f64
1,6.0,"""a""","""x""",6.0
2,7.0,"""b""","""y""",-2.0
null,null,"""d""","""z""",-3.0
3,8.0,"""c""",null,null


In [3]:
df = pl.DataFrame({
    "a": [10,20,30,40,50],
    "b": ["x","y","z","w","v"]
})

indices = [2, 0, 4]             # arbitrary order
filtered_df = df[indices]
filtered_df

a,b
i64,str
30,"""z"""
10,"""x"""
50,"""v"""


In [4]:
# %% [markdown]
# ## Quantile Benchmark: NumPy vs Polars
# This cell measures the time to compute the 0.1 (10th percentile) quantile for each column
# in a 2D NumPy array versus a Polars DataFrame.

# %% [code]
import os
os.environ["POLARS_MAX_THREADS"] = "1"
import numpy as np
import polars as pl
import pandas as pd
import timeit
print(pl.thread_pool_size())

# Setup
N = 100_000   # number of rows
M = 100          # number of columns
arr = np.random.rand(N, M)
df = pl.DataFrame({f"col{i}": arr[:, i] for i in range(M)})

reps = 10

# NumPy quantile timing
np_q_time = timeit.timeit(
    'np.quantile(arr, 0.1, axis=0)',
    globals=globals(),
    number=reps
) / reps

# Polars quantile timing
pl_q_time = timeit.timeit(
    'df = pl.DataFrame({f"col{i}": arr[:, i] for i in range(M)}); df.quantile(0.1)',
    globals=globals(),
    number=reps
) / reps

# Display results
results = pd.DataFrame({
    'Library': ['NumPy', 'Polars'],
    'Avg Time (s)': [np_q_time, pl_q_time]
})
print(results)


16
  Library  Avg Time (s)
0   NumPy      0.278182
1  Polars      0.100574


In [5]:
# %% [markdown]
# ## Concatenation Benchmark: NumPy vs Polars

# %% [code]
import numpy as np
import polars as pl
import pandas as pd
import timeit

# Setup data
N = 200_000
arr_list = [np.random.rand(N) for _ in range(200)]

reps = 50

# NumPy concatenation
np_cat_time = timeit.timeit('np.concatenate(arr_list)', globals=globals(), number=reps) / reps

# Polars concatenation
pl_cat_time = timeit.timeit('df_list = [pl.DataFrame({"col": arr}) for arr in arr_list]; pl.concat(df_list)', globals=globals(), number=reps) / reps

# Show results
results = pd.DataFrame({
    'Library': ['NumPy', 'Polars'],
    'Avg Time (s)': [np_cat_time, pl_cat_time]
})
print(results)


  Library  Avg Time (s)
0   NumPy      0.086293
1  Polars      0.003414


In [6]:
import numpy as np
import polars as pl
import pandas as pd
import timeit

# Setup data
N = 100_000
arr = np.random.rand(N)
val = 0.5
# Benchmark repetitions
reps = 20

# NumPy timing
np_time = timeit.timeit('arr[arr > val]', globals=globals(), number=reps) / reps

# Polars DataFrame filter timing
pl_time = timeit.timeit('df = pl.DataFrame({"col": arr}); df.filter(pl.col("col") > val); arr2 = df.to_numpy()', globals=globals(), number=reps) / reps

# Prepare results
results = pd.DataFrame({
    'Library': ['NumPy', 'Polars'],
    'Avg Time (s)': [np_time, pl_time]
})

print(results)


  Library  Avg Time (s)
0   NumPy      0.000502
1  Polars      0.000489


In [7]:
# %%
import polars as pl
from datetime import date

def __add_target(meta_pl_all: pl.DataFrame, days_After: int = 2):
    meta_pl_all = meta_pl_all.with_row_index("row_index")
    meta_pl_all = meta_pl_all.with_columns(
        (pl.col("date") + pl.duration(days=days_After)).alias("target_date")
    )
    meta_pl_all = meta_pl_all.sort(["ticker", "date"])
    prices = (
        meta_pl_all
        .select(["ticker","date","Close"])
        .rename({"date":"price_date","Close":"price"})
    )
    return (
        meta_pl_all
        .join_asof(
            prices,
            left_on="target_date",
            right_on="price_date",
            by="ticker",
            strategy="backward",
        )
        .with_columns(pl.col("price").alias("target_close"))
        .drop(["price_date","price","target_date","row_index"])
    )

# %%
# 1) Simple case: matching next-day price
df1 = pl.DataFrame({
    "ticker": ["A","A"],
    "date":    [date(2025,5,1), date(2025,5,3)],
    "Close":   [10.0, 12.0]
})
print(__add_target(df1, days_After=2))

# %%
# 2) No future price available (edge: gap too large)
df2 = pl.DataFrame({
    "ticker": ["B"],
    "date":    [date(2025,5,1)],
    "Close":   [5.0]
})
print(__add_target(df2, days_After=3))
# expect target_close = null

# %%
# 3) Multiple tickers and duplicate dates
df3 = pl.DataFrame({
    "ticker": ["A","A","B","B"],
    "date":    [date(2025,5,1), date(2025,5,2), date(2025,5,1), date(2025,5,4)],
    "Close":   [1,2,3,4]
})
print(__add_target(df3, days_After=1))


shape: (2, 4)
┌────────┬────────────┬───────┬──────────────┐
│ ticker ┆ date       ┆ Close ┆ target_close │
│ ---    ┆ ---        ┆ ---   ┆ ---          │
│ str    ┆ date       ┆ f64   ┆ f64          │
╞════════╪════════════╪═══════╪══════════════╡
│ A      ┆ 2025-05-01 ┆ 10.0  ┆ 12.0         │
│ A      ┆ 2025-05-03 ┆ 12.0  ┆ 12.0         │
└────────┴────────────┴───────┴──────────────┘
shape: (1, 4)
┌────────┬────────────┬───────┬──────────────┐
│ ticker ┆ date       ┆ Close ┆ target_close │
│ ---    ┆ ---        ┆ ---   ┆ ---          │
│ str    ┆ date       ┆ f64   ┆ f64          │
╞════════╪════════════╪═══════╪══════════════╡
│ B      ┆ 2025-05-01 ┆ 5.0   ┆ 5.0          │
└────────┴────────────┴───────┴──────────────┘
shape: (4, 4)
┌────────┬────────────┬───────┬──────────────┐
│ ticker ┆ date       ┆ Close ┆ target_close │
│ ---    ┆ ---        ┆ ---   ┆ ---          │
│ str    ┆ date       ┆ i64   ┆ i64          │
╞════════╪════════════╪═══════╪══════════════╡
│ A      ┆ 2025-05

C:\Users\kimer\AppData\Local\Temp\ipykernel_20056\2338872466.py:18: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  .join_asof(


In [8]:
import numpy as np

def check_features(npz_path, meta_key, feat_key, names_key, value_range=None):
    data = np.load(npz_path,allow_pickle=True )
    meta   = data[meta_key]
    feats  = data[feat_key]
    names  = data[names_key]

    # non-empty
    assert meta.size > 0,      f"{meta_key} is empty"
    assert feats.size > 0,     f"{feat_key} is empty"
    assert names.size > 0,     f"{names_key} is empty"

    # column counts
    assert meta.shape[0] == feats.shape[0], (
        f"{meta_key}.shape[0]={meta.shape[0]} ≠ {feat_key}.shape[0]={feats.shape[0]}"
    )
    
    def assert_no_empty(arr: np.array, name):
        # numeric arrays: check NaN
        if np.issubdtype(arr.dtype, np.number):
            assert np.all(np.isfinite(arr)), f"{name} contains NaN or infinities"
        # object/string arrays: check empty or None
        else:
            bad = [i for i, v in enumerate(arr.ravel()) if v is None or v == ""]
            assert not bad, f"{name} has empty entries at flat indices {bad}"

    # tree block
    if "time" in feat_key:
        assert_no_empty(data[feat_key], feat_key)
    
    if "tree" in feat_key:
        assert feats.shape[1] == len(names), (
            f"{feat_key}.shape[1]={feats.shape[1]} ≠ len({names_key})={len(names)}"
        )

    if "time" in feat_key:
        assert feats.shape[2] == len(names), (
            f"{feat_key}.shape[2]={feats.shape[2]} ≠ len({names_key})={len(names)}"
        )

    # optional value-range check
    if value_range:
        lo, hi = value_range
        assert np.all(feats >= lo) and np.all(feats <= hi), (
            f"{feat_key} values not in [{lo}, {hi}]"
        )

# paths
label = 2015
group = "group_snp500_finanTo2011"
tree_npz = "../src/featureAlchemy/bin/TreeFeatures_{}_{}.npz".format(label, group)
time_npz = "../src/featureAlchemy/bin/TimeFeatures_meta_{}_{}.npz".format(label, group)

# run checks
#check_features(tree_npz,
#               meta_key="meta_tree",
#               feat_key="treeFeatures",
#               names_key="treeFeaturesNames")

check_features(time_npz,
               meta_key="meta_time",
               feat_key="timeFeatures",
               names_key="timeFeaturesNames",
               value_range=(0.0, 1.0))

print("All feature files look good!")  


FileNotFoundError: [Errno 2] No such file or directory: '../src/featureAlchemy/bin/TimeFeatures_meta_2015_group_snp500_finanTo2011.npz'

In [ ]:
time_npz = f"../src/featureAlchemy/bin/TimeFeatures_meta_{label}_{group}.npz"
data     = np.load(time_npz, allow_pickle=True)
met = data['meta_time']
feats    = data['timeFeatures']
names    = data['timeFeaturesNames']

# ─── Value-range check ─────────────────────────────────────────────────────────
lo, hi = 0.0, 1.0

# reshape so each column is a feature
flat = feats.reshape(-1, feats.shape[-1])

# compute per-column min/max
mins = flat.min(axis=0)
maxs = flat.max(axis=0)

# find offending features
bad = [
    (i, names[i], float(mins[i]), float(maxs[i]))
    for i in range(len(names))
    if mins[i] < lo or maxs[i] > hi
]

if bad:
    print(f"{len(bad)} feature(s) out of [{lo}, {hi}]:")
    for idx, name, mn, mx in bad:
        print(f" • #{idx} {name}: min={mn:.3g}, max={mx:.3g}")
else:
    print("All timeFeatures ∈ [0,1]")

4 feature(s) out of [0.0, 1.0]:
 • #9 Fourier_Price_RSMERatioCoeff_1_MH_1: min=-0.383, max=0.462
 • #10 Fourier_Price_RSMERatioCoeff_2_MH_1: min=-0.326, max=0.462
 • #11 Fourier_Price_RSMERatioCoeff_3_MH_1: min=-0.247, max=0.463
 • #12 Fourier_Price_RSMERatioCoeff_4_MH_1: min=-0.246, max=0.463


In [ ]:
num_sam_tr = 60  # matrix size
n_diags = 4      # number of off-diagonals on each side
A_sparse = SpecialMatrix.boundedDiagonal_oneNetCol(num_sam_tr, n_diags)

# For demonstration: print the sparse matrix in dense form.
A_full = A_sparse.todense()

# --- 3. Print the matrices ---
print("\nFull dense matrix A_full:\n", A_full[0])
#print("\nSparse matrix A_sparse:")
#print(A_sparse)

# --- 4. Matrix-vector multiplication ---
x = np.random.rand(num_sam_tr)
y_full = A_full @ x
y_sparse = A_sparse @ x

print("All are close:", np.allclose(y_full, y_sparse))  # True


Full dense matrix A_full:
 [[0.2        0.16666667 0.14285714 0.125      0.11111111 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]
All are close: True


In [ ]:
import scipy.sparse.linalg as spla


num_sam_tr = 8  # matrix size
n_diags = 3      # number of off-diagonals on each side
A_sparse = SpecialMatrix.boundedDiagonal_sparse(num_sam_tr, n_diags)

# For demonstration: print the sparse matrix in dense form.
A_full = A_sparse.todense()

## --- 5. Inverse matrix-vector multiplication ---
x = np.random.rand(num_sam_tr)
## Solve A*y = x for y.
y_inv_full = np.linalg.solve(A_full, x)
y_inv_sparse = spla.spsolve(A_sparse, x)
print("\nInverse matrix-vector multiplication (solving A*y = x):")
print("Absolute difference:\n", np.mean(np.abs(y_inv_full - y_inv_sparse)))

# --- Compute numerical rank and determinant for the dense matrix A_full ---
U, s, Vh = np.linalg.svd(A_full)
tol = max(A_full.shape) * np.amax(s) * np.finfo(s.dtype).eps
num_rank_full = np.sum(s > tol)
det_full = np.linalg.det(A_full)
print("Dense matrix:")
print("  Numerical rank:", num_rank_full)
print("  Determinant:", det_full)


Inverse matrix-vector multiplication (solving A*y = x):
Absolute difference:
 2600040400926568.0
Dense matrix:
  Numerical rank: 7
  Determinant: 2.0706540538643876e-17


matrix([[ 0.45067453],
        [-0.14372927],
        [-0.20424094],
        [-0.54761511],
        [ 0.49338753],
        [ 0.45388748],
        [-0.31895834],
        [-0.18340588]])

In [ ]:
import numpy as np

def circulant_multiply(c, x):
    """Multiply circulant matrix (defined by first column c) by vector x."""
    return np.real(np.fft.ifft(np.fft.fft(c) * np.fft.fft(x)))

def circulant_inverse_multiply(c, b):
    """Solve Ax = b for circulant matrix A defined by first column c."""
    eigenvalues = np.fft.fft(c)
    print(f"Lowest eigenvalue: {np.min(np.abs(eigenvalues))}")
    if np.any(np.abs(eigenvalues) < 1e-12):
        raise ValueError("Circulant matrix is singular!")
    return np.real(np.fft.ifft(np.fft.fft(b) / eigenvalues))

def circulant_coeffs(n, a):
    # Compute cyclic distances: d[k] = min(k, n-k)
    d = np.minimum(np.arange(n), n - np.arange(n))
    # Exponential decay: main diagonal is 1, neighbors decay as exp(-a*d)
    coeffs = np.exp(-a * d)
    # Normalize so the coefficients sum to 1
    return coeffs / coeffs.sum()
# Example usage:
num_sam_tr = 20
c = circulant_coeffs(num_sam_tr, 0.5)  # first column of the circulant matrix
eps = 1e-6 # to avoid singular matrix
c[0] -= 1 - eps
x = np.random.rand(num_sam_tr)  # vector to multiply

# Multiply circulant matrix by x
y = circulant_multiply(c, x)
print("Result of circulant multiply:", y)

# Now, solve Ax = y to recover x
x_recovered = circulant_inverse_multiply(c, y)
print("Original x:", x)
print("Recovered x:", x_recovered)

Result of circulant multiply: [ 0.21012606 -0.13314346  0.15737162  0.02087521 -0.03788862 -0.15362854
  0.15314803 -0.16258518  0.37174242 -0.10257611 -0.03736177 -0.26782281
 -0.06814705  0.04932619  0.29093784 -0.02471498 -0.33416127  0.04366359
  0.22688358 -0.20203418]
Lowest eigenvalue: 9.999999999177334e-07
Original x: [0.24748827 0.60572137 0.29671893 0.45556372 0.54210204 0.67540687
 0.34681953 0.68082847 0.12293884 0.66881797 0.64903305 0.91469166
 0.68131546 0.51208106 0.23120074 0.58238626 0.92121761 0.4864869
 0.25784285 0.69901605]
Recovered x: [0.24748827 0.60572137 0.29671893 0.45556372 0.54210204 0.67540687
 0.34681953 0.68082847 0.12293884 0.66881797 0.64903305 0.91469166
 0.68131546 0.51208106 0.23120074 0.58238626 0.92121761 0.4864869
 0.25784285 0.69901605]


In [10]:
num_sam_tr = 50000
num_sam_te = 1000
diagonal_num = 0
n_feat = 10
sam_tr = np.zeros((num_sam_tr, n_feat), dtype=float)
sam_te = np.zeros((num_sam_te, n_feat), dtype=float)

for i in range(n_feat):
    sam_tr[:,i] = np.random.normal(loc=0-i/5000, scale=2+i/3000, size=num_sam_tr) #train
    sam_te[:,i] = np.random.normal(loc=0+i/5000, scale=1-i/3000, size=num_sam_te) #test
sam_te[:,-3] = np.random.normal(loc=0+0.3, scale=1, size=num_sam_te) + np.random.normal(loc=-0.3, scale=1.2, size=num_sam_te)
sam_te[:,-2] = np.random.uniform(low=-0.5, high=0.5, size=num_sam_te)
sam_te[:,-1] = np.random.uniform(low=-1.9, high=1.9, size=num_sam_te)

argsort_sam_tr = np.argsort(sam_tr, axis=0)
argsort_sam_te = np.argsort(sam_te, axis=0)
re_argsort_sam_tr = np.argsort(argsort_sam_tr, axis=0)
re_argsort_sam_te = np.argsort(argsort_sam_te, axis=0)
sort_sam_tr = np.sort(sam_tr, axis=0)
sort_sam_te = np.sort(sam_te, axis=0)

ksDist = DistributionTools.ksDistance(sam_tr, sam_te, weights=None)
print(ksDist.mean())

importance = np.zeros(n_feat)
importance[0] = 1
s = DistributionTools.establishMatchingWeight(sam_tr, sam_te, 10, 0.4)
#s = weights @ importance

ksDist_s = DistributionTools.ksDistance(sam_tr, sam_te, weights=s)
print(ksDist_s.mean())

fig, axs = plt.subplots(n_feat, 1, figsize=(6, 4 * n_feat))
for cube_idx in range(n_feat):
    # sort s according to the training samples for this cube_idx
    s_sorted = s[argsort_sam_tr[:, cube_idx]]
    s_cum = np.cumsum(s_sorted) / np.sum(s_sorted)
    
    axs[cube_idx].scatter(sort_sam_tr[:, cube_idx], s_cum,
                           label='S Distribution', marker='.', edgecolors='blue')
    axs[cube_idx].scatter(sort_sam_te[:, cube_idx],
                           np.linspace(0, 1, num_sam_te),
                           label='Test Distribution', marker='.', edgecolors='red')
    axs[cube_idx].set_title(f'Sample Distribution (cube_idx = {cube_idx})')
    axs[cube_idx].set_xlabel('x')
    axs[cube_idx].set_ylabel('Density')
    axs[cube_idx].legend()

plt.legend()
plt.tight_layout()
plt.show()
    

0.04678486138395458
0.018274742528420162


In [5]:
ksDist_s

array([0.32109053, 0.34814003, 0.325643  , 0.09236356, 0.70709246,
       0.3234183 ])

In [20]:
import bisect

def count_entries_between(v, w):
    lo = np.searchsorted(w, v[:-1], side="right")
    hi = np.searchsorted(w, v[1:], side="left")
    return hi - lo

# Example usage:
v = [1, 3, 5, 10]
w = [2, 4, 6, 7, 9]
print(count_entries_between(v, w))

w=np.array(w).astype(int)


[1 1 3]


In [51]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
num_sam_tr = 1000
# Generate samples from a normal distribution
data = np.random.normal(loc=0, scale=1, size=num_sam_tr)

# Create random weights and normalize them to sum to 1
weights = np.random.rand(num_sam_tr)+0.5
weights /= weights.sum()

# Sort data and associated weights
sorted_indices = np.argsort(data)
data_sorted = data[sorted_indices]
weights_sorted = weights[sorted_indices]

# Compute the weighted cumulative sum (CDF)
cdf = np.cumsum(weights_sorted)

# Plot the weighted CDF
plt.step(data_sorted, cdf, where='post')
plt.xlabel('Data')
plt.ylabel('Weighted CDF')
plt.title('Weighted CDF for Normal Distribution')
plt.show()

In [71]:
num_sam2 = 100000
num_sam1 = 100000//10
sam1 = np.random.normal(loc=0, scale=1, size=num_sam1) #test
sam2 = np.random.normal(loc=0, scale=2, size=num_sam2) #train

argsort_sam1 = np.argsort(sam1)
argsort_sam2 = np.argsort(sam2)
sort_sam1 = sam1[argsort_sam1]
sort_sam2 = sam2[argsort_sam2]

#diff_sort_sam1 = np.diff(sort_sam1)
#diff_sort_sam2 = np.diff(sort_sam2)
#diff_xpts = np.diff(xpts)
#
#quot_sam1 = diff_xpts / diff_sort_sam1
#quot_sam2 = diff_xpts / diff_sort_sam2
#
#quot = quot_sam1 / quot_sam2

def count_entries_between(v, w):
    lo = np.searchsorted(w, v[:-1], side="right")
    hi = np.searchsorted(w, v[1:], side="left")
    return hi - lo
def smooth_array(data, window_size=3):
    # Create a uniform window
    window = np.ones(window_size) / window_size
    # Convolve while keeping the output length the same
    return np.convolve(data, window, mode='same')

weights2 = count_entries_between(sort_sam2, sort_sam1)
weights2 = smooth_array(weights2, window_size = 10)
#weights2 = np.ones(num_sam2)
weights2 = np.insert(weights2, 0, weights2[0])
weights2 *= num_sam2/np.sum(weights2)

weights0 = np.ones(num_sam2)
cdf2 = np.cumsum(weights2)
cdf2 = cdf2 / cdf2[-1]
weights2_org = weights2[np.argsort(argsort_sam1)]
plt.plot(weights2)
plt.show()

print(np.cumsum(weights2)[-1])
print(np.cumsum(weights0)[-1])
print(np.sum(weights0))
# Plot the weighted CDF
plt.step(sort_sam2, cdf2, where='post', color='green')
plt.xlabel('Data')
plt.ylabel('Weighted CDF')
plt.title('Weighted Cumulative Distribution Function')

#isam1 = np.searchsorted(sort_sam1, xpts)

plt.scatter(sort_sam1, np.linspace(0, 1, num_sam1), label='Sam test', marker='.', linestyle='--', linewidths=0.1, edgecolors='blue')
plt.scatter(sort_sam2, np.linspace(0, 1, num_sam2), label='Sam train', marker='.', linestyle='--', linewidths=0.1, edgecolors='red')
plt.title('samples')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.show()

100000.0
100000.0
100000.0
